###  **Question 3: IAQF Academic Competition**


*Imports*

In [26]:
import pandas as pd 
import numpy as np
import yfinance as yf
from datetime import date
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

##### **a) What are the implications of the noted concentration for that strategy?**

**Data**

*Reading*

In [27]:
sector_etfs = {
    'Baseline': {
        "Large-Cap": "SPY",
        "Small-Cap": "^SP600"
    },
    "Technology": {
        "Large-Cap": "XLK",
        "Small-Cap": "PSCT"
    },
    "Healthcare": {
        "Large-Cap": "XLV",
        "Small-Cap": "PSCH"
    },
    "Financials": {
        "Large-Cap": "XLF",
        "Small-Cap": "PSCF"
    },
    "Consumer Discretionary": {
        "Large-Cap": "XLY",
        "Small-Cap": "PSCD"
    },
    "Consumer Staples": {
        "Large-Cap": "XLP",
        "Small-Cap": "PSCC"
    },
    "Industrials": {
        "Large-Cap": "XLI",
        "Small-Cap": "PSCI"
    },
    "Energy": {
        "Large-Cap": "XLE",
        "Small-Cap": "PSCE"
    },
    "Materials": {
        "Large-Cap": "XLB",
        "Small-Cap": "PSCM"
    },
    "Utilities": {
        "Large-Cap": "XLU",
        "Small-Cap": "PSCU"
    },
    "Real Estate": {
        "Large-Cap": "XLRE",
        "Small-Cap": "ROOF"
    }
}

In [28]:
tickers = [_ for x in sector_etfs.values() for _ in x.values()]
start_date = date(2010,1,1)
end_date = date(2025,1,1)

# ETF Returns
prices = yf.download(tickers = tickers,start = start_date,end = end_date)['Close']

ff_data = pd.read_csv('../extra-data/FF_3Factor.CSV',skiprows = 4,index_col=0)
ff5_data = pd.read_csv('../extra-data/FF_5Factor.csv',index_col=0)

[*********************100%***********************]  22 of 22 completed


*Cleaning*

In [29]:
returns = prices.pct_change().dropna()

# Clean FF Data
ff_data = ff_data.iloc[:-1]
ff_data.index = pd.to_datetime(ff_data.index).date

ff5_data.index = pd.to_datetime(ff5_data.index).date

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_60581/1153697555.py:1: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



##### *i)* Baseline Strategy


Baseline strategy is underperforming, show how this is connected to tech exposure

In [30]:
baseline_strategy = pd.DataFrame(returns['^SP600'] - returns['SPY'],columns=['daily_returns'])
baseline_strategy['cumulative_returns'] = baseline_strategy['daily_returns'].cumsum()

##### *ii)* Industry Sector Basis

Show the strategy performance at a sector basis

In [31]:
sector_baseline_returns = {}

for sector,(large,small) in sector_etfs.items():
    sector_baseline_returns[sector] = (returns[sector_etfs[sector][small]] - returns[sector_etfs[sector][large]]).cumsum()

sector_baseline_returns = pd.DataFrame(sector_baseline_returns)

##### *iii)* Risk Attribution (Fama-French)

In [32]:
import statsmodels.api as sm
# Merge datasets
df = baseline_strategy.join(ff5_data, how='inner')

# Calculate excess returns
df['Excess_Return'] = df['daily_returns'] - df['rf']

# Run regression
X = df[['smb', 'hml', 'rmw', 'cma','umd']]
X = sm.add_constant(X)  # Add intercept
y = df['Excess_Return']

model = sm.OLS(y, X).fit()
print(model.summary())

# Extract risk contribution
betas = model.params[1:]  # Exclude intercept
factor_vols = df[['smb', 'hml', 'rmw', 'cma','umd']].std()  # Factor standard deviations

# Compute risk contribution
risk_contributions = (betas**2 * factor_vols**2) / sum(betas**2 * factor_vols**2)
print("Risk Attribution:\n", risk_contributions)

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     2087.
Date:                Fri, 28 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:28:40   Log-Likelihood:                 3557.5
No. Observations:                 753   AIC:                            -7103.
Df Residuals:                     747   BIC:                            -7075.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003   7.88e-05     -3.639      0.0

#### **Plots**

*i)* Baseline Strategy Performance

In [33]:
returns['SPY']

Date
2015-10-09    0.000596
2015-10-12    0.000944
2015-10-13   -0.006302
2015-10-14   -0.004794
2015-10-15    0.015355
                ...   
2024-12-24    0.011115
2024-12-26    0.000067
2024-12-27   -0.010527
2024-12-30   -0.011412
2024-12-31   -0.003638
Name: SPY, Length: 2322, dtype: float64

In [34]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = baseline_strategy.index,
        y = baseline_strategy['cumulative_returns'],
        name = 'Benchmark Strategy',
    )
)

fig.add_trace(
    go.Scatter(
        x = returns.index,
        y = returns['SPY'].cumsum(),
        name = 'S&P 500 Large Cap'
    )
)

fig.add_trace(
    go.Scatter(
        x = returns.index,
        y = returns['^SP600'].cumsum(),
        name = 'S&P 600 Small Cap'
    )
)

fig.update_layout(title = 'Benchmark Strategy Returns')
fig.update_layout(
                  showlegend=True,
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  width = 800,height = 550,
                  xaxis_title = 'Date',
                  xaxis=dict(title_standoff=3),
                  template = 'plotly_white'
                )
fig.update_yaxes(title = 'Cumulative Simple Returns')
fig.update_xaxes(title = 'Date')
fig.show()

*ii)* Strategy at Sector Level

In [35]:
fig = go.Figure()


for strategy in sector_baseline_returns:

    fig.add_trace(
        go.Scatter(
            x = sector_baseline_returns.index,
            y = sector_baseline_returns[strategy],
            name = strategy
        )
    )

fig.update_layout(title = 'Strategy Sector Level Returns')
fig.update_layout(
                  showlegend=True,
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  width = 800,height = 550,
                  xaxis_title = 'Date',
                  xaxis=dict(title_standoff=3),
                  template = 'plotly_white'
                )
fig.update_yaxes(title = 'Cumulative Simple Returns')
fig.update_xaxes(title = 'Date')
fig.show()

In [36]:
ff_data.index = pd.to_datetime(ff_data.index)

In [37]:
ff_data['Year'] = ff_data.index.year

In [38]:
ff_yearly = ff_data.groupby('Year').agg({"SMB":'sum'}).reset_index()

In [39]:
ff_yearly = ff_yearly[ff_yearly['Year'] >= 2018]

In [40]:
ff_yearly.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n & Year & SMB \\\\\n\\midrule\n92 & 2018 & -3.080000 \\\\\n93 & 2019 & -4.570000 \\\\\n94 & 2020 & 12.020000 \\\\\n95 & 2021 & -2.390000 \\\\\n96 & 2022 & -6.290000 \\\\\n97 & 2023 & -1.250000 \\\\\n98 & 2024 & -8.190000 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [41]:

fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = ff_yearly['Year'],
        y = ff_yearly['SMB']
    )
)

fig.show()

##### **b) How would you modify the trading strategy to correct for any implication you may have found?**

In [42]:
small_cap = ['PSCT', 'PSCH', 'PSCF','PSCD', 'PSCC', 'PSCI', 'PSCE', 'PSCM','PSCU','ROOF']
large_cap = ['XLK', 'XLV', 'XLF', 'XLY', 'XLP', 'XLI', 'XLE', 'XLB', 'XLU', 'XLRE']

start_date = date(2015,1,1)
end_date = date(2025,1,1)

data = yf.download(tickers = small_cap + large_cap,start = start_date, end = end_date)['Close']

returns = data.pct_change().dropna()

[*********************100%***********************]  20 of 20 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_60581/3441130964.py:9: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [43]:
class Momentum:

    def __init__(self, data, returns, lookback, rebalance, type = 'Equal'):
        # Inputs
        self.data = data
        self.returns = returns
        self.lookback = lookback
        self.rebalance = rebalance
        self.type = type

        # Stats
        self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
        self.portfolio_returns = pd.Series(0.0, index=self.returns.index, dtype=float)

    def run(self):

        for t in range(self.lookback, len(self.returns.index)):
            # Compute portfolio return for the day
            daily_return = (self.weights * self.returns.iloc[t]).sum()
            self.portfolio_returns.iloc[t] = daily_return

            if t % self.rebalance == 0:

                # Get window returns
                small_cap_window_returns = self.returns.iloc[t - self.lookback:t,][small_cap]
                large_cap_window_returns = self.returns.iloc[t - self.lookback:t,][large_cap]

                # Calculate momentum, long and short threshold
                momentum_small_cap = self.signal(small_cap_window_returns)
                momentum_large_cap = self.signal(large_cap_window_returns)

                # Select assets in the top and bottom deciles
                long_assets = momentum_small_cap.nlargest(5).index
                short_assets = momentum_large_cap.nsmallest(5).index

                # Equally Weighted Portfolio
                self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
                self.weights[long_assets] = 1 / len(long_assets) if len(long_assets) > 0 else 0
                self.weights[short_assets] = -1 / len(short_assets) if len(short_assets) > 0 else 0

            

        # Clean + Compute Stats
        self._clean_returns()
        self._compute_stats()

    def signal(self, window_returns):
        """Calculate Momentum Signals"""
        if self.type == 'Equal':

            # Calculate momentum (cumulative returns over the lookback period)
            momentum = window_returns.sum() / window_returns.std()
    
            return momentum
        
    def _clean_returns(self):
        """Clean Portfolio Returns"""

        _df = pd.DataFrame(self.portfolio_returns,columns=['daily_return'])
        _df['cumulative_return'] = _df['daily_return'].cumsum()
        
        self.portfolio_returns = _df

    def _compute_stats(self):
        """Compute Portfolio Stats"""
        
        # Annualized Return
        annual_ret = self.portfolio_returns['daily_return'].mean()*252

        # Volatility
        annual_vol = self.portfolio_returns['daily_return'].std()*np.sqrt(252)

        # Sharpe Ratio
        sharpe_ratio = annual_ret / annual_vol

        self.stats = {'Sharpe Ratio': round(sharpe_ratio,2),
                      'Annual Return': round(annual_ret,2),
                        'Annual Volatility': round(annual_vol,2)}

In [44]:
m = Momentum(data,returns,120,30,'Equal')
m.run()
portfolio_returns = m.portfolio_returns

##### **c) Can you provide a backtest comparison between the standard strategy and the modified strategy?**

In [45]:
baseline_strategy.iloc[:120,0] = 0

In [46]:
baseline_strategy['cumulative_returns'] = baseline_strategy['daily_returns'].cumsum()

In [47]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = portfolio_returns.index[120:],
        y = portfolio_returns.iloc[120:,:]['cumulative_return'],
        name = 'Improved Strategy'
    )
)

fig.add_trace(
    go.Scatter(
        x = baseline_strategy.index[120:],
        y = baseline_strategy.iloc[120:,:]['cumulative_returns'],
        name = 'Benchmark Strategy',
    )
)


fig.update_layout(title = 'Improved Strategy vs Benchmark',
                  showlegend=True,
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  width = 800,height = 550,
                  xaxis_title = 'Date',
                  xaxis=dict(title_standoff=3),
                  template = 'plotly_white'
                )
fig.update_yaxes(title = 'Cumulative Simple Return')
fig.update_xaxes(title = 'Date')
fig.show()

In [48]:
def calculate_summary_stats(daily_returns):

    # Annualized return
    ann_return = np.prod(1 + daily_returns) ** (252 / len(daily_returns)) - 1

    # Annualized volatility
    ann_vol = np.std(daily_returns, ddof=1) * np.sqrt(252)

    # Sharpe ratio
    excess_returns = daily_returns
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns, ddof=1) * np.sqrt(252)

    # Sortino ratio (downside deviation)
    downside_returns = daily_returns[daily_returns < 0]
    downside_vol = np.std(downside_returns, ddof=1) * np.sqrt(252)
    sortino_ratio = np.mean(excess_returns) / downside_vol if downside_vol != 0 else np.nan

    # Maximum drawdown
    cumulative_returns = (1 + daily_returns).cumprod()
    peak = np.maximum.accumulate(cumulative_returns)
    drawdown = (cumulative_returns - peak) / peak
    max_dd = drawdown.min()

    return {
        "Annualized Return": round(ann_return,4),
        "Annualized Volatility": round(ann_vol,4),
        "Sharpe Ratio": round(sharpe_ratio,4),
        "Sortino Ratio": round(sortino_ratio,4),
        "Maximum Drawdown": round(max_dd,4),
    }


In [49]:
improve1_stats = calculate_summary_stats(portfolio_returns.iloc[120:,:]['daily_return'])
baseline_stats = calculate_summary_stats(baseline_strategy.iloc[120:,:]['daily_returns'])


pd.DataFrame({"Baseline Strategy":baseline_stats,'Improvement 1':improve1_stats}).to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n & Baseline Strategy & Improvement 1 \\\\\n\\midrule\nAnnualized Return & -0.049700 & -0.013600 \\\\\nAnnualized Volatility & 0.128200 & 0.129200 \\\\\nSharpe Ratio & -0.333700 & -0.041400 \\\\\nSortino Ratio & -0.002100 & -0.000200 \\\\\nMaximum Drawdown & -0.455100 & -0.316100 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [70]:
import statsmodels.api as sm
# Merge datasets
df = portfolio_returns.join(ff5_data, how='inner')

# Calculate excess returns
df['Excess_Return'] = df['daily_return'] - df['rf']

# Run regression
X = df[['smb', 'hml', 'rmw', 'cma','umd']]
X = sm.add_constant(X)  # Add intercept
y = df['Excess_Return']

model = sm.OLS(y, X).fit()
print(model.summary())

# Extract risk contribution
betas = model.params[1:]  # Exclude intercept
factor_vols = df[['smb', 'hml', 'rmw', 'cma','umd']].std()  # Factor standard deviations

# Compute risk contribution
risk_contributions = (betas**2 * factor_vols**2) / sum(betas**2 * factor_vols**2)
print("Risk Attribution:\n", risk_contributions)

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     229.3
Date:                Tue, 25 Feb 2025   Prob (F-statistic):          3.34e-148
Time:                        10:35:03   Log-Likelihood:                 2808.8
No. Observations:                 753   AIC:                            -5606.
Df Residuals:                     747   BIC:                            -5578.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001      0.000     -0.574      0.5

### **Improvements 2**